In [7]:
import ee
import folium

In [1]:
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [2]:
geometry = ee.Geometry.Point([31.5051, -26.6061])

#location = roi.geometry().centroid().coordinates().getInfo()[::-1]
location = geometry.coordinates().getInfo()[::-1]

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
rgbVis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
}

In [3]:
# Cloud masking
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])

In [4]:
filtered = s2.filter(ee.Filter.date('2020-01-01', '2020-05-01')) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.bounds(geometry)) \
  .map(maskS2clouds)

In [5]:
# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

In [6]:
withNdvi = filtered.map(addNDVI)
median = withNdvi.median()

## Map Result

In [8]:
# Define a method for displaying Earth Engine image tiles to folium map.
def addLayer (self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

In [9]:
# Add EE drawing method to folium.
folium.Map.addLayer = addLayer

# Create the map object.
Map = folium.Map(location=location, zoom_start = 10)

In [10]:
Map.addLayer(median, rgbVis, 'Sentinel')

#Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

#Display the map.
display(Map)